In [1]:
!pip install langchain openai weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [21]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-proj-AD8Av3qEYHvSVRwY5TsGT3BlbkFJHRoClOryP5RZWgeASOSZ'


In [28]:
import pandas as pd

csv_file_path = '/content/drive/MyDrive/merged_df.csv'
df = pd.read_csv(csv_file_path)
df = df.iloc[:200]
text_file_path = 'main_doc.txt'
with open(text_file_path, 'w', encoding='utf-8') as file:
    for index, row in df.iterrows():
        line = ", ".join([f"{col}: {row[col]}" for col in df.columns])
        file.write(line + "\n")

print("Data has been successfully written to the text file.")


Data has been successfully written to the text file.


In [29]:
from langchain.document_loaders import TextLoader

loader = TextLoader('/content/main_doc.txt')
documents = loader.load()

In [30]:
!pip install tiktoken

In [31]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [32]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

embedded weaviate is already listening on port 8079
Embedded weaviate wasn't listening on ports http:8079 & grpc:50060, so starting embedded weaviate again
Embedded weaviate wasn't listening on ports http:8079 & grpc:50060, so starting embedded weaviate again
Started /root/.cache/weaviate-embedded: process ID 11465
Started /root/.cache/weaviate-embedded: process ID 11464


In [33]:
retriever = vectorstore.as_retriever()

In [45]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "Describe Saint Florian"
rag_chain.invoke(query)


'Saint Florian is a painting by Francesco del Cossa, created between 1473 and 1474. It was commissioned by Floriano Griffoni for a family chapel in the church of San Petronio, Bologna. The painting was later moved to the Casa Aldrovandi in Bologna and then purchased by Joseph Spiridon in 1881. It was eventually sold to the Samuel H. Kress Foundation in May 1936 and gifted to the National Gallery of Art in 1939.'